In [13]:
import pandas as pd
import geopandas as gpd
import os

PROCESSING US POST OFFICES DATA

In [14]:
print(os.getcwd())

c:\Users\AnaLíviadosSantosMaz\Documents


In [15]:
# reading the file

arquivo = "data/us-post-offices-random-coords.csv"

us_post_offices_raw = pd.read_csv(arquivo)

us_post_offices_raw

C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel_11836\1540659858.py:5: DtypeWarning: Columns (5,6,11) have mixed types. Specify dtype option on import or set low_memory=False.
  us_post_offices_raw = pd.read_csv(arquivo)


,Name,AltName,OrigName,State,County1,County2,County3,OrigCounty,Established,Discontinued,...,GNIS.Feature.Class,GNIS.OrigName,GNIS.OrigCounty,GNIS.Latitude,GNIS.Longitude,GNIS.ELEV_IN_M,GNIS.Dist,Latitude,Longitude,RandomCoordsFlag
0,A AND M COLLEGE,NaN,A AND M COLLEGE,MS,OKTIBBEHA,NaN,NaN,Oktibbeha,1922.0,1932.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.508712,-88.703363,True
1,A. B. C.,NaN,A. B. C.,TN,SUMNER,NaN,NaN,Sumner,1880.0,1914.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.419931,-86.563870,True
2,AARON,NaN,AARON,MO,BATES,NaN,NaN,Bates,1895.0,1933.0,...,Post Office,Aaron Post Office (historical),Bates,38.422222,-94.154167,268.0,1.0,38.422222,-94.154167,False
3,AARON,NaN,AARON,TN,BENTON,NaN,NaN,Benton,1889.0,1890.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.205923,-88.094459,True
4,AARON,NaN,AARON,TN,BRADLEY,NaN,NaN,Bradley,1860.0,1860.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.143874,-84.931121,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166135,TRUE,NaN,TRUE,OK,HARMON,NaN,NaN,Harmon,1901.0,1905.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.738756,-99.782970,True
166136,TRUE,NaN,TRUE,TX,YOUNG,NaN,NaN,Young,1894.0,1912.0,...,Populated Place,True,Young,33.279830,-98.728670,359.0,1.0,33.279830,-98.728670,False
166137,SIERRA BLANCA,NaN,SIERRA BLANCA,TX,HUDSPETH,NaN,NaN,Hudspeth,1882.0,NaN,...,Post Office,Sierra Blanca Post Office,Hudspeth,31.175930,-105.360000,1380.0,1.0,31.175930,-105.360000,False
166138,GRAHAM'S MILLS,GRAHAMS MILLS,GRAHAM'S MILLS,TX,SHELBY,NaN,NaN,Shelby,1854.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.962916,-94.393446,True


In [ ]:
# Filtering out the Post Office with ID 127138 (Bexar, Texas), as it appears to be an outlier.
# The recorded establishment year is 1779, which is unusual since Texas was not part of the US at that time.
# This data point is likely incorrect and does not align with the historical population expansion
# and establishment of Post Offices in the US during that period.

us_post_offices_raw = us_post_offices_raw[us_post_offices_raw['ID'] != 127138]

In [16]:
# getting only the columns that are needed

us_post_offices_raw_cleaned = us_post_offices_raw.loc[:,['Name','GNIS.State', 'State', 'County1', 'GNIS.OrigCounty','Established','Discontinued','Continuous','Duration','Latitude','Longitude','RandomCoordsFlag']]


# replacing null values with zero
us_post_offices_raw_cleaned['Established'] = us_post_offices_raw_cleaned['Established'].fillna(0).astype(int)
us_post_offices_raw_cleaned['Discontinued'] = us_post_offices_raw_cleaned['Discontinued'].fillna(0).astype(int)

us_post_offices_raw_cleaned

,Name,GNIS.State,State,County1,GNIS.OrigCounty,Established,Discontinued,Continuous,Duration,Latitude,Longitude,RandomCoordsFlag
0,A AND M COLLEGE,NaN,MS,OKTIBBEHA,NaN,1922,1932,True,10.0,33.508712,-88.703363,True
1,A. B. C.,NaN,TN,SUMNER,NaN,1880,1914,True,34.0,36.419931,-86.563870,True
2,AARON,MO,MO,BATES,Bates,1895,1933,True,38.0,38.422222,-94.154167,False
3,AARON,NaN,TN,BENTON,NaN,1889,1890,True,1.0,36.205923,-88.094459,True
4,AARON,NaN,TN,BRADLEY,NaN,1860,1860,True,0.0,35.143874,-84.931121,True
...,...,...,...,...,...,...,...,...,...,...,...,...
166135,TRUE,NaN,OK,HARMON,NaN,1901,1905,True,4.0,34.738756,-99.782970,True
166136,TRUE,TX,TX,YOUNG,Young,1894,1912,True,18.0,33.279830,-98.728670,False
166137,SIERRA BLANCA,TX,TX,HUDSPETH,Hudspeth,1882,0,True,NaN,31.175930,-105.360000,False
166138,GRAHAM'S MILLS,NaN,TX,SHELBY,NaN,1854,0,True,NaN,31.962916,-94.393446,True


In [17]:
# adjusting null values by combining two complementary columns

us_post_offices_raw_cleaned['GNIS.State'] = us_post_offices_raw_cleaned['GNIS.State'].fillna(us_post_offices_raw_cleaned['State'])
us_post_offices_raw_cleaned['GNIS.OrigCounty'] = us_post_offices_raw_cleaned['GNIS.OrigCounty'].fillna(us_post_offices_raw_cleaned['County1'])

us_post_offices_raw_cleaned

,Name,GNIS.State,State,County1,GNIS.OrigCounty,Established,Discontinued,Continuous,Duration,Latitude,Longitude,RandomCoordsFlag
0,A AND M COLLEGE,MS,MS,OKTIBBEHA,OKTIBBEHA,1922,1932,True,10.0,33.508712,-88.703363,True
1,A. B. C.,TN,TN,SUMNER,SUMNER,1880,1914,True,34.0,36.419931,-86.563870,True
2,AARON,MO,MO,BATES,Bates,1895,1933,True,38.0,38.422222,-94.154167,False
3,AARON,TN,TN,BENTON,BENTON,1889,1890,True,1.0,36.205923,-88.094459,True
4,AARON,TN,TN,BRADLEY,BRADLEY,1860,1860,True,0.0,35.143874,-84.931121,True
...,...,...,...,...,...,...,...,...,...,...,...,...
166135,TRUE,OK,OK,HARMON,HARMON,1901,1905,True,4.0,34.738756,-99.782970,True
166136,TRUE,TX,TX,YOUNG,Young,1894,1912,True,18.0,33.279830,-98.728670,False
166137,SIERRA BLANCA,TX,TX,HUDSPETH,Hudspeth,1882,0,True,NaN,31.175930,-105.360000,False
166138,GRAHAM'S MILLS,TX,TX,SHELBY,SHELBY,1854,0,True,NaN,31.962916,-94.393446,True


In [18]:
us_post_offices_raw_cleaned['GNIS.OrigCounty'] = us_post_offices_raw_cleaned['GNIS.OrigCounty'].str.upper()

In [19]:
us_post_offices_raw_cleaned = us_post_offices_raw_cleaned.loc[
    (us_post_offices_raw_cleaned['GNIS.OrigCounty'] != 'UNKNOWN') & 
    (us_post_offices_raw_cleaned['GNIS.OrigCounty'].notna())
]

In [20]:
complete_years = range(1639, 2001)

In [21]:
# calculating number of post offices opened

established_table = us_post_offices_raw_cleaned[['Name','GNIS.OrigCounty','GNIS.State','Established']]

established_table = established_table.groupby(['GNIS.OrigCounty','GNIS.State','Established'])['Name'].count().reset_index(name='Opened Post Offices')

established_table = established_table.rename(columns={'Established': 'Year'})

established_table

,GNIS.OrigCounty,GNIS.State,Year,Opened Post Offices
0,?,UT,1864,2
1,?,VA,1822,1
2,??,NC,1868,1
3,??,NC,1882,1
4,ABBEVILLE,SC,1792,1
...,...,...,...,...
97446,ZIEBACH,SD,1905,1
97447,ZIEBACH,SD,1910,2
97448,ZIEBACH,SD,1911,6
97449,ZIEBACH,SD,1913,1


In [ ]:
# grouping the data by 'GNIS.State', 'GNIS.OrigCounty', and 'Year', and summing the 'Opened Post Offices'
grouped = established_table.groupby(['GNIS.State', 'GNIS.OrigCounty', 'Year'])['Opened Post Offices'].sum().reset_index()

# defining the range of years from 1639 to 2000
complete_years = range(1639, 2001)  # year 2000 inclusive

# creating a DataFrame with all possible combinations of 'State', 'County', and 'Year'
states_and_counties = established_table[['GNIS.State', 'GNIS.OrigCounty']].drop_duplicates()
all_combinations = pd.MultiIndex.from_product(
    [states_and_counties['GNIS.State'].unique(), 
     states_and_counties['GNIS.OrigCounty'].unique(), 
     complete_years], 
    names=['GNIS.State', 'GNIS.OrigCounty', 'Year']
)

# reindexing the 'grouped' DataFrame to ensure all combinations of 'State', 'County', and 'Year' exist
grouped_full = grouped.set_index(['GNIS.State', 'GNIS.OrigCounty', 'Year']).reindex(all_combinations, fill_value=0).reset_index()

# displaying the final result
established_table_optimized = grouped_full
established_table_optimized

,GNIS.State,GNIS.OrigCounty,Year,Opened Post Offices
0,UT,?,1639,0
1,UT,?,1640,0
2,UT,?,1641,0
3,UT,?,1642,0
4,UT,?,1643,0
...,...,...,...,...
36837115,MI/OH,ZIEBACH,1996,0
36837116,MI/OH,ZIEBACH,1997,0
36837117,MI/OH,ZIEBACH,1998,0
36837118,MI/OH,ZIEBACH,1999,0


In [22]:
#calculating number of post offices closed

discontinued_table = us_post_offices_raw_cleaned[['Name','GNIS.OrigCounty', 'GNIS.State','Discontinued']]

discontinued_table = discontinued_table.groupby(['GNIS.OrigCounty', 'GNIS.State', 'Discontinued'])['Name'].count().reset_index(name='Closed Post Offices')

discontinued_table = discontinued_table.rename(columns={'Discontinued': 'Year'})

discontinued_table

,GNIS.OrigCounty,GNIS.State,Year,Closed Post Offices
0,?,UT,1865,1
1,?,UT,1867,1
2,?,VA,1823,1
3,??,NC,1910,1
4,??,NC,1931,1
...,...,...,...,...
93588,ZIEBACH,SD,1924,1
93589,ZIEBACH,SD,1935,1
93590,ZIEBACH,SD,1943,1
93591,ZIEBACH,SD,1960,1


In [23]:
# grouping the data by 'GNIS.State', 'GNIS.OrigCounty', and 'Year', and summing the 'Closed Post Offices'
grouped = discontinued_table.groupby(['GNIS.State', 'GNIS.OrigCounty', 'Year'])['Closed Post Offices'].sum().reset_index()

# defining the range of years from 1639 to 2000
complete_years = range(1639, 2001)  # Year 2000 inclusive

# creating a DataFrame with all possible combinations of 'State', 'County', and 'Year'
states_and_counties = discontinued_table[['GNIS.State', 'GNIS.OrigCounty']].drop_duplicates()
all_combinations = pd.MultiIndex.from_product(
    [states_and_counties['GNIS.State'].unique(), 
     states_and_counties['GNIS.OrigCounty'].unique(), 
     complete_years], 
    names=['GNIS.State', 'GNIS.OrigCounty', 'Year']
)

# reindexing the 'grouped' DataFrame to ensure all combinations of 'State', 'County', and 'Year' exist
grouped_full = grouped.set_index(['GNIS.State', 'GNIS.OrigCounty', 'Year']).reindex(all_combinations, fill_value=0).reset_index()

# displaying the final result
discontinued_table_optimized = grouped_full
discontinued_table_optimized


,GNIS.State,GNIS.OrigCounty,Year,Closed Post Offices
0,UT,?,1639,0
1,UT,?,1640,0
2,UT,?,1641,0
3,UT,?,1642,0
4,UT,?,1643,0
...,...,...,...,...
36837115,MI/OH,ZIEBACH,1996,0
36837116,MI/OH,ZIEBACH,1997,0
36837117,MI/OH,ZIEBACH,1998,0
36837118,MI/OH,ZIEBACH,1999,0


In [ ]:
# doing a merge to put in just one table all the opens and closes for each post county

df_merged = pd.merge(established_table_optimized, discontinued_table_optimized, on=['GNIS.OrigCounty', 'GNIS.State', 'Year'], how='outer')

df_merged['Opened Post Offices'] = df_merged['Opened Post Offices'].fillna(0).astype('int')

df_merged['Closed Post Offices'] = df_merged['Closed Post Offices'].fillna(0).astype('int')

df_merged

,GNIS.State,GNIS.OrigCounty,Year,Opened Post Offices,Closed Post Offices
0,AK,?,1639,0,0
1,AK,?,1640,0,0
2,AK,?,1641,0,0
3,AK,?,1642,0,0
4,AK,?,1643,0,0
...,...,...,...,...,...
36837115,WY,ZIEBACH,1996,0,0
36837116,WY,ZIEBACH,1997,0,0
36837117,WY,ZIEBACH,1998,0,0
36837118,WY,ZIEBACH,1999,0,0


In [25]:
 #adding column that does the "balance" between closed and opened post offices

df_merged['Balance'] = df_merged['Opened Post Offices'] - df_merged['Closed Post Offices']

df_merged = df_merged.sort_values(by='Year')

df_merged = df_merged[df_merged['Year'] != 0]

df_merged

,GNIS.State,GNIS.OrigCounty,Year,Opened Post Offices,Closed Post Offices,Balance
0,AK,?,1639,0,0,0
29660470,WI,SCHLEY,1639,0,0,0
35414822,VA,WEXFORD,1639,0,0,0
2663596,TX,BELTRAMI,1639,0,0,0
27336430,TN,RANKIN,1639,0,0,0
...,...,...,...,...,...,...
24491833,NC,ORANGE,2000,0,0,0
24492195,ND,ORANGE,2000,0,0,0
24492557,NE,ORANGE,2000,0,0,0
24493281,NJ,ORANGE,2000,0,0,0


In [29]:
# dropping the 'Opened Post Offices' and 'Closed Post Offices' columns from the merged DataFrame
df_merged_aggregated = df_merged.drop(columns=['Opened Post Offices', 'Closed Post Offices'])

df_merged_aggregated


,GNIS.State,GNIS.OrigCounty,Year,Balance
0,AK,?,1639,0
29660470,WI,SCHLEY,1639,0
35414822,VA,WEXFORD,1639,0
2663596,TX,BELTRAMI,1639,0
27336430,TN,RANKIN,1639,0
...,...,...,...,...
24491833,NC,ORANGE,2000,0
24492195,ND,ORANGE,2000,0
24492557,NE,ORANGE,2000,0
24493281,NJ,ORANGE,2000,0


In [30]:
# grouping the data by 'Year', 'GNIS.OrigCounty', and 'GNIS.State', summing the 'Balance' column
# and renaming the resulting column to 'Number of Post Offices'
df_merged_aggregated_over_year = df_merged_aggregated.groupby(
    ['Year', 'GNIS.OrigCounty', 'GNIS.State']
)['Balance'].sum().reset_index(name='Number of Post Offices')

df_merged_aggregated_over_year


,Year,GNIS.OrigCounty,GNIS.State,Number of Post Offices
0,1639,?,AK,0
1,1639,?,AL,0
2,1639,?,AR,0
3,1639,?,AZ,0
4,1639,?,CA,0
...,...,...,...,...
36837115,2000,ZIEBACH,VT,0
36837116,2000,ZIEBACH,WA,0
36837117,2000,ZIEBACH,WI,0
36837118,2000,ZIEBACH,WV,0


In [31]:
# calculating the cumulative sum of 'Number of Post Offices' for each 'GNIS.OrigCounty' and 'GNIS.State',
# and creating a new column 'Cumulative Post Offices' to store the result
df_merged_aggregated_over_year['Cumulative Post Offices'] = df_merged_aggregated_over_year.groupby(
    ['GNIS.OrigCounty', 'GNIS.State']
)['Number of Post Offices'].cumsum()

df_merged_aggregated_over_year

,Year,GNIS.OrigCounty,GNIS.State,Number of Post Offices,Cumulative Post Offices
0,1639,?,AK,0,0
1,1639,?,AL,0,0
2,1639,?,AR,0,0
3,1639,?,AZ,0,0
4,1639,?,CA,0,0
...,...,...,...,...,...
36837115,2000,ZIEBACH,VT,0,0
36837116,2000,ZIEBACH,WA,0,0
36837117,2000,ZIEBACH,WI,0,0
36837118,2000,ZIEBACH,WV,0,0


In [33]:
# filtering the DataFrame to include only rows where:
# the 'Year' is between 1790 and 2000 (inclusive),
# the 'Year' is a multiple of 10 (to capture data at 10-year intervals).
df_merged_aggregated_over_year = df_merged_aggregated_over_year[
    (df_merged_aggregated_over_year['Year'] >= 1790) &
    (df_merged_aggregated_over_year['Year'] <= 2000) &
    (df_merged_aggregated_over_year['Year'] % 10 == 0)
]

# displaying the filtered DataFrame
df_merged_aggregated_over_year


,Year,GNIS.OrigCounty,GNIS.State,Number of Post Offices,Cumulative Post Offices
15365760,1790,?,AK,0,0
15365761,1790,?,AL,0,0
15365762,1790,?,AR,0,0
15365763,1790,?,AZ,0,0
15365764,1790,?,CA,0,0
...,...,...,...,...,...
36837115,2000,ZIEBACH,VT,0,0
36837116,2000,ZIEBACH,WA,0,0
36837117,2000,ZIEBACH,WI,0,0
36837118,2000,ZIEBACH,WV,0,0


SHAPEFILE COUNTIES GEOMETRY

In [34]:
gdf = gpd.read_file("C:/Users/AnaLíviadosSantosMaz/Documents/counties_states_combined_simplified.geojson")

gdf.head(10)

,STATE_COUNTY,STATE,CWA,COUNTYNAME,FIPS,TIME_ZONE,FE_AREA,LON,LAT,geometry
0,AK_Aleutians East,AK,AFC,Aleutians East,02013,A,None,-161.8631,55.4039,"MULTIPOLYGON (((-162.45576 54.36005, -162.4963..."
1,AK_Aleutians West,AK,AFC,Aleutians West,02016,Ah,None,-111.2564,52.8883,"MULTIPOLYGON (((-179.08545 51.29422, -179.0581..."
2,AK_Anchorage,AK,AFC,Anchorage,02020,A,None,-149.1067,61.1501,"MULTIPOLYGON (((-149.2379 61.45531, -149.1876 ..."
3,AK_Bethel,AK,AFC,Bethel,02050,A,None,-159.7733,60.9210,"MULTIPOLYGON (((-166.10333 59.83039, -166.107 ..."
4,AK_Bristol Bay,AK,AFC,Bristol Bay,02060,A,None,-156.7013,58.7423,"POLYGON ((-156.31814 58.89495, -156.31767 58.6..."
5,AK_Chugach,AK,AFC,Chugach,02063,A,None,-145.7950,60.7357,"MULTIPOLYGON (((-147.81645 60.06182, -147.8676..."
6,AK_Copper River,AK,AFC,Copper River,02066,A,None,-143.9685,61.8844,"POLYGON ((-145.1489 63.22221, -145.1484 63.133..."
7,AK_Denali,AK,AFG,Denali,02068,A,None,-150.0083,63.6739,"POLYGON ((-150.7497 64.36511, -147.9969 64.341..."
8,AK_Dillingham,AK,AFC,Dillingham,02070,A,None,-158.2119,59.8021,"MULTIPOLYGON (((-159.97017 58.61057, -159.9606..."
9,AK_Fairbanks North Star,AK,AFG,Fairbanks North Star,02090,A,None,-146.5630,64.8084,"POLYGON ((-146.1646 65.44291, -146.1123 65.403..."


In [35]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3284 entries, 0 to 3283
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   STATE_COUNTY  3284 non-null   object  
 1   STATE         3284 non-null   object  
 2   CWA           3284 non-null   object  
 3   COUNTYNAME    3284 non-null   object  
 4   FIPS          3284 non-null   object  
 5   TIME_ZONE     3284 non-null   object  
 6   FE_AREA       3102 non-null   object  
 7   LON           3284 non-null   float64 
 8   LAT           3284 non-null   float64 
 9   geometry      3284 non-null   geometry
dtypes: float64(2), geometry(1), object(7)
memory usage: 256.7+ KB


In [36]:
gdf = gdf[['COUNTYNAME', 'STATE', 'geometry']]

gdf

,COUNTYNAME,STATE,geometry
0,Aleutians East,AK,"MULTIPOLYGON (((-162.45576 54.36005, -162.4963..."
1,Aleutians West,AK,"MULTIPOLYGON (((-179.08545 51.29422, -179.0581..."
2,Anchorage,AK,"MULTIPOLYGON (((-149.2379 61.45531, -149.1876 ..."
3,Bethel,AK,"MULTIPOLYGON (((-166.10333 59.83039, -166.107 ..."
4,Bristol Bay,AK,"POLYGON ((-156.31814 58.89495, -156.31767 58.6..."
...,...,...,...
3279,Sweetwater,WY,"POLYGON ((-109.7512 42.26571, -107.5227 42.261..."
3280,Teton,WY,"POLYGON ((-110.7457 44.66641, -110.6679 44.666..."
3281,Uinta,WY,"POLYGON ((-110.9245 41.57921, -110.048 41.5774..."
3282,Washakie,WY,"POLYGON ((-107.87469 44.16821, -107.1286 44.16..."


In [37]:
gdf['COUNTYNAME'] = gdf['COUNTYNAME'].str.upper()

MERGE BETWEEN STATES GEOMETRY AND US POST OFFICE DATABASE OVER YEAR

In [ ]:
df_merged_post_offices_geometry = pd.merge(
    df_merged_aggregated_over_year, 
    gdf, 
    left_on=['GNIS.State', 'GNIS.OrigCounty'], 
    right_on=['STATE', 'COUNTYNAME'],
    how='left'
)

df_merged_post_offices_geometry

,Year,GNIS.OrigCounty,GNIS.State,Number of Post Offices,Cumulative Post Offices,COUNTYNAME,STATE,geometry
0,1790,?,AK,0,0,NaN,NaN,None
1,1790,?,AL,0,0,NaN,NaN,None
2,1790,?,AR,0,0,NaN,NaN,None
3,1790,?,AZ,0,0,NaN,NaN,None
4,1790,?,CA,0,0,NaN,NaN,None
...,...,...,...,...,...,...,...,...
2238715,2000,ZIEBACH,VT,0,0,NaN,NaN,None
2238716,2000,ZIEBACH,WA,0,0,NaN,NaN,None
2238717,2000,ZIEBACH,WI,0,0,NaN,NaN,None
2238718,2000,ZIEBACH,WV,0,0,NaN,NaN,None


In [39]:
df_merged_post_offices_geometry = df_merged_post_offices_geometry.drop(columns=['COUNTYNAME', 'STATE', 'Number of Post Offices'])

df_merged_post_offices_geometry

,Year,GNIS.OrigCounty,GNIS.State,Cumulative Post Offices,geometry
0,1790,?,AK,0,None
1,1790,?,AL,0,None
2,1790,?,AR,0,None
3,1790,?,AZ,0,None
4,1790,?,CA,0,None
...,...,...,...,...,...
2238715,2000,ZIEBACH,VT,0,None
2238716,2000,ZIEBACH,WA,0,None
2238717,2000,ZIEBACH,WI,0,None
2238718,2000,ZIEBACH,WV,0,None


In [40]:
df_merged_post_offices_geometry = df_merged_post_offices_geometry[df_merged_post_offices_geometry['geometry'].notna()]

df_merged_post_offices_geometry

,Year,GNIS.OrigCounty,GNIS.State,Cumulative Post Offices,geometry
147,1790,ABBEVILLE,SC,0,"POLYGON ((-82.26109 34.42061, -82.24689 34.409..."
177,1790,ACADIA,LA,0,"MULTIPOLYGON (((-92.6215 30.09121, -92.62379 3..."
258,1790,ACCOMACK,VA,0,"MULTIPOLYGON (((-75.93307 37.58204, -75.9323 3..."
278,1790,ADA,ID,0,"POLYGON ((-116.4183 43.80731, -116.2818 43.806..."
330,1790,ADAIR,IA,0,"POLYGON ((-94.6569 41.50411, -94.24149 41.5036..."
...,...,...,...,...,...
2238511,2000,YUMA,AZ,6,"POLYGON ((-113.946 33.37761, -113.334 33.37731..."
2238513,2000,YUMA,CO,9,"POLYGON ((-102.0515 40.41811, -102.0505 39.574..."
2238605,2000,ZAPATA,TX,3,"POLYGON ((-99.3336 27.27321, -98.9554 27.26941..."
2238658,2000,ZAVALA,TX,3,"POLYGON ((-99.41679 29.09131, -99.4087 28.6407..."


In [42]:
import datetime
df_merged_post_offices_geometry['Date'] = df_merged_post_offices_geometry['Year'].apply(
    lambda x: datetime.datetime(x, 1, 1))

df_merged_post_offices_geometry

C:\Users\AnaLíviadosSantosMaz\AppData\Local\Temp\ipykernel_11836\495281809.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_post_offices_geometry['Date'] = df_merged_post_offices_geometry['Year'].apply(


,Year,GNIS.OrigCounty,GNIS.State,Cumulative Post Offices,geometry,Height 3D Map,Date
147,1790,ABBEVILLE,SC,0,"POLYGON ((-82.26109 34.42061, -82.24689 34.409...",0.0,1790-01-01
177,1790,ACADIA,LA,0,"MULTIPOLYGON (((-92.6215 30.09121, -92.62379 3...",0.0,1790-01-01
258,1790,ACCOMACK,VA,0,"MULTIPOLYGON (((-75.93307 37.58204, -75.9323 3...",0.0,1790-01-01
278,1790,ADA,ID,0,"POLYGON ((-116.4183 43.80731, -116.2818 43.806...",0.0,1790-01-01
330,1790,ADAIR,IA,0,"POLYGON ((-94.6569 41.50411, -94.24149 41.5036...",0.0,1790-01-01
...,...,...,...,...,...,...,...
2238511,2000,YUMA,AZ,6,"POLYGON ((-113.946 33.37761, -113.334 33.37731...",216.0,2000-01-01
2238513,2000,YUMA,CO,9,"POLYGON ((-102.0515 40.41811, -102.0505 39.574...",729.0,2000-01-01
2238605,2000,ZAPATA,TX,3,"POLYGON ((-99.3336 27.27321, -98.9554 27.26941...",27.0,2000-01-01
2238658,2000,ZAVALA,TX,3,"POLYGON ((-99.41679 29.09131, -99.4087 28.6407...",27.0,2000-01-01


In [43]:
df_merged_post_offices_geometry.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67672 entries, 147 to 2238709
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Year                     67672 non-null  int64         
 1   GNIS.OrigCounty          67672 non-null  object        
 2   GNIS.State               67672 non-null  object        
 3   Cumulative Post Offices  67672 non-null  int32         
 4   geometry                 67672 non-null  geometry      
 5   Height 3D Map            67672 non-null  float64       
 6   Date                     67672 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), geometry(1), int32(1), int64(1), object(2)
memory usage: 3.9+ MB


In [44]:
df_merged_post_offices_geometry = gpd.GeoDataFrame(
    df_merged_post_offices_geometry,
    geometry='geometry' 
)

In [45]:
df_merged_post_offices_geometry.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 67672 entries, 147 to 2238709
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Year                     67672 non-null  int64         
 1   GNIS.OrigCounty          67672 non-null  object        
 2   GNIS.State               67672 non-null  object        
 3   Cumulative Post Offices  67672 non-null  int32         
 4   geometry                 67672 non-null  geometry      
 5   Height 3D Map            67672 non-null  float64       
 6   Date                     67672 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), geometry(1), int32(1), int64(1), object(2)
memory usage: 3.9+ MB


In [46]:
df_merged_post_offices_geometry.to_file('tests_project_us_post_offices_counties/merged_post_offices_over_years.geojson', driver='GeoJSON')

In [47]:
df_merged_post_offices_geometry.to_csv('tests_project_us_post_offices_counties/merged_post_offices_geometry_over_years.csv',index=False, encoding='utf-8')

In [48]:
df_merged_post_offices_geometry.to_excel('tests_project_us_post_offices_counties/merged_post_offices_geometry_over_years.xlsx')